In [2]:
#converting 

from io import StringIO
col = ['tweet_id','sentiment', 'content']
df = df[col]
df = df[pd.notnull(df['content'])]
# df = df.head(10000)
df.columns = ['tweet_id','sentiment', 'content']
df['category_id'] = df['sentiment'].factorize()[0]
category_id_df = df[['sentiment', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'sentiment']].values)
df.head()

,tweet_id,sentiment,content,category_id
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...,0
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,1
2,1956967696,sadness,Funeral ceremony...gloomy friday...,1
3,1956967789,enthusiasm,wants to hang out with friends SOON!,2
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,3


In [3]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
df.groupby('sentiment').content.count().plot.bar(ylim=0)
plt.show()

<Figure size 800x600 with 1 Axes>

In [4]:
import re
from nltk.corpus import stopwords
engStopWords = stopwords.words("english")

def removeIrrelvantInfo(df,replace_user_reference='<user>',replace_webpage='<url>',replace_hashtag='<hashtag>'):
    regex_user_reference = re.compile('@[a-zA-Z0-9]+')
    references = []
#     for entry in df['content']:
#         references.append(regex_references.findall(entry))
    df['content'] = df['content'].str.replace(regex_user_reference,replace_user_reference)

    regex_websites = re.compile('http://[www.]*[a-zA-Z0-9]+.[a-z]+/[a-zA-Z0-9//]*')
    websites = []
#     for entry in df['content']:
#         regex_websites.findall(entry)
    df['content'] = df['content'].str.replace(regex_websites,replace_webpage)
    regex_extrachars = re.compile('[\.\:]+')
    df['content'] = df['content'].str.replace(regex_extrachars,'')
    df['content'] = df['content'].str.lower()
    df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in engStopWords]))
    df['content'] = df['content'].str.replace(r"n't", " not ")
    df['content'] = df['content'].str.replace(r"?", " ? ")
    df['content'] = df['content'].str.replace(r"!", " ! ")
    return df

In [5]:
print(df.head())
df = removeIrrelvantInfo(df)
print(df.head())

     tweet_id   sentiment                                            content  \
0  1956967341       empty  @tiffanylue i know  i was listenin to bad habi...   
1  1956967666     sadness  Layin n bed with a headache  ughhhh...waitin o...   
2  1956967696     sadness                Funeral ceremony...gloomy friday...   
3  1956967789  enthusiasm               wants to hang out with friends SOON!   
4  1956968416     neutral  @dannycastillo We want to trade with someone w...   

   category_id  
0            0  
1            1  
2            1  
3            2  
4            3  
     tweet_id   sentiment                                            content  \
0  1956967341       empty  <user> know listenin bad habit earlier started...   
1  1956967666     sadness             layin n bed headache ughhhhwaitin call   
2  1956967696     sadness                      funeral ceremonygloomy friday   
3  1956967789  enthusiasm                         wants hang friends soon !    
4  1956968416    

In [6]:
import numpy as np
# embeddings_text = open('glove50.txt')
WordToVec       = dict()
with open('./../../../glove.twitter.27B.50d.txt') as embeddings_text: 
    for line in embeddings_text:
        entry_array = line.split(" ")
        word        = entry_array[0] 
        word_vector = np.array(entry_array[1:],dtype='float32')
        WordToVec[word] = word_vector

In [7]:
print(WordToVec.get("<user>"))

[ 0.78704   0.72151   0.29148  -0.056527  0.31683   0.47172   0.023461
  0.69568   0.20782   0.60985  -0.22386   0.7481   -2.6208    0.20117
 -0.48104   0.12897   0.035239 -0.24486  -0.36088   0.026686  0.28978
 -0.10698  -0.34621   0.021053  0.54514  -1.0958   -0.274     0.2233
  1.0827   -0.029018 -0.84029   0.58619  -0.36511   0.34016   0.89615
  0.32757   0.24267   0.68404  -0.34374   0.13583  -2.2162   -0.42537
  0.46157   0.88626  -0.22014   0.025599 -0.38615   0.080107 -0.075323
 -0.61461 ]


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras import optimizers

Using TensorFlow backend.


In [9]:
max_features = 8000
tokenizer = Tokenizer(num_words=max_features,lower=True, split=' ')
tokenizer.fit_on_texts(df['content'].values)
print(tokenizer.document_count)
X = tokenizer.texts_to_sequences(df['content'].values)
X = pad_sequences(X)
X.shape
X[1]


40000


array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
       194,  79, 329, 197], dtype=int32)

In [10]:
# from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer(stop_words='english',max_features=8000)

# X_train_counts = count_vect.fit_transform(df['content'].values)
# print(X_train_counts.shape)

In [11]:
# vocab=len(tokenizer.word_index.items())
# print(vocab)
print(tokenizer.word_index.items())

dict_items([('ichat', 29788), ("kk's", 25071), ('breed', 8902), ('usd70', 31652), ('pessimist', 26968), ("ta'3yeem", 33929), ("s'en", 20555), ('4rom', 13584), ('replys', 7415), ('action', 1646), ("meani'm", 23396), ('haz', 6387), ('nope', 667), ('trolled', 15541), ('porter', 7826), ('jelly', 4047), ('fightin', 10594), ('rearry', 26358), ('distorted', 18579), ('dodd', 18103), ('yoot', 14668), ('tapped', 18578), ('buy', 338), ('thursbut', 19027), ('unmuted', 17546), ('lonesome', 9979), ("noriega's", 31671), ('repierced', 11264), ('inferno', 17693), ('people', 77), ('yoou', 34276), ('another', 130), ('interlock', 26641), ("queen's", 32879), ('hahahhah', 27183), ('puppyyy', 34792), ('geodefense', 34350), ('yeaah', 12142), ('mentionedmeh', 27056), ('bannished', 21215), ('eft', 24535), ('ogberry', 22854), ('anit', 18233), ('shallow', 5532), ('explosions', 30805), ('toysonly', 21934), ('kairos', 19203), ('geordanos', 20327), ('omelette', 11160), ('badoptus', 14409), ('sixer', 23948), ('mommas

In [12]:
embedding_matrix = np.zeros((max_features, 50))

for word, index in tokenizer.word_index.items():
    if index > max_features-1:
        continue
    else:
        embedding_vector = WordToVec.get(word.lower())
#         print(embedding_vector)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [13]:
embedding_matrix[15]

array([-0.16140001,  0.22660001,  0.039414  ,  0.33642   , -0.62880999,
       -0.1035    ,  0.89679003,  0.72547001, -0.91096002,  0.066873  ,
       -0.88328999, -0.10026   , -3.98510003, -0.17095999, -0.77985001,
        0.74011999, -0.57288003, -0.23543   , -1.22300005,  0.83262002,
        0.15536   ,  0.75707   , -1.06980002,  0.75106001,  0.24905001,
        0.73247999, -0.13189   , -0.74685001,  1.55799997, -0.35084999,
       -0.41599   , -0.22845   ,  0.0049928 ,  0.57968998,  0.15524   ,
        0.41604999,  0.14427   ,  0.23541   ,  0.069321  ,  0.86312997,
       -2.20830011, -0.95704001,  0.57643998,  0.55662   ,  0.077452  ,
        0.10875   , -0.39614999, -0.81149   , -0.43867999,  0.063696  ])

In [14]:
embed_dim = 50
lstm_out = 32

adam = optimizers.Adam(lr=0.01)
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model = Sequential()
# model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1], weights=[embedding_matrix], trainable=False))
# model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(50,activation='relu'))
model.add(Dense(13,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer=sgd,metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            400000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dense_1 (Dense)              (None, 13)                429       
Total params: 411,053
Trainable params: 11,053
Non-trainable params: 400,000
_________________________________________________________________
None


In [31]:
Y = pd.get_dummies(df['category_id']).values
xhat=X[100+1:]
print(X.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(40000, 30)
(32000, 30) (32000, 13)
(8000, 30) (8000, 13)


In [19]:
validation_size = 1000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

In [20]:
batch_size = 32
history=model.fit(X_train, Y_train, epochs = 500, batch_size=batch_size, verbose = 2,validation_data=(X_validate,Y_validate))

ValueError: Error when checking target: expected dense_1 to have shape (13,) but got array with shape (1,)

In [19]:

score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 2.82
acc: 0.30
